In [14]:
# En esta celda se puede probar la funcion de analisis de una imagen

from lib.image import image_data
from lib.analysis import dyssynchonia_analysis
import cv2

# Path completo donde esta la imagen (incluyendo nombre y extencion)
path = 'photos_examples/c1d000.tif'

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20

image = image_data.get_image_data(path)
# Select ROI
fromCenter = False
showCrosshair = False
seleted_parameters = cv2.selectROI(image, fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.waitKey(0)
cv2.destroyAllWindows()


image = image_data.crop_vertical(image, x_start, x_end)
image = image_data.crop_horizontal(image, y_start, y_end)

dyssynchonia_analysis.analyze_image(image, min_dist_between_maxs, ancho_corte)

{'image': {'intensities': array([4214, 4156, 4074, 3995, 3908, 3854, 3775, 3710, 3630, 3572, 3502,
         3451, 3374, 3317, 3250, 3191, 3122, 3078, 3007, 2948, 2902, 2840,
         2790, 2754, 2686, 2652, 2615, 2574, 2528, 2486, 2449, 2439, 2454,
         2506, 2608, 2759, 2939, 3130, 3358, 3572, 3816, 4065, 4326, 4589,
         4842, 5072, 5312, 5546, 5734, 5932, 6083, 6171, 6224, 6229, 6220,
         6179, 6122, 6064, 5997, 5916, 5838, 5742, 5648, 5555, 5460, 5374,
         5265, 5180, 5068, 4957, 4872, 4767, 4678, 4603, 4500, 4416, 4329,
         4251, 4163, 4097, 4006, 3937, 3851, 3792, 3706, 3641, 3578, 3499,
         3437, 3358, 3301, 3237, 3174, 3126, 3055, 2995, 2930, 2890, 2821,
         2782, 2733, 2674, 2628, 2570, 2513, 2474, 2428, 2369, 2328, 2291,
         2287, 2334, 2441, 2586, 2755, 2959, 3169, 3398, 3652, 3914, 4178,
         4442, 4716, 4988, 5243, 5496, 5728, 5934, 6108, 6226, 6280, 6296,
         6291, 6265, 6227, 6170, 6107, 6033, 5968, 5894, 5815, 5724, 5624,
 

In [9]:
# En esta celda se puede probar la funcion de analisis todas las imagenes .tif en un directorio
import os
from lib.image import image_data
from lib.analysis import dyssynchonia_analysis
import cv2

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20


# Path donde estan las imagenes
path = "photos_examples"

images_paths = [ f'{path}/{file}' for file in os.listdir(path) if file.endswith(".tif") ]
images = [ image_data.get_image_data(image_path) for image_path in images_paths ]


In [10]:
fromCenter = False
showCrosshair = False

#Tomamos la primer imagen para seleccionar el recorte elegido el resto de las imagenes se van a recortar igual
seleted_parameters = cv2.selectROI(images[0], fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
images = [ image_data.crop_vertical(image, x_start, x_end) for image in images ]
images = [ image_data.crop_horizontal(image, y_start, y_end) for image in images ]
images

[array([[15, 16, 16, ..., 21, 21, 21],
        [16, 16, 16, ..., 22, 22, 22],
        [16, 16, 16, ..., 23, 23, 23],
        ...,
        [18, 18, 18, ..., 28, 28, 28],
        [18, 19, 19, ..., 29, 29, 29],
        [19, 20, 20, ..., 31, 30, 31]], dtype=uint8),
 array([[33, 33, 33, ..., 25, 25, 25],
        [32, 32, 32, ..., 24, 24, 24],
        [31, 31, 31, ..., 24, 24, 24],
        ...,
        [27, 27, 27, ..., 21, 21, 21],
        [27, 27, 27, ..., 21, 21, 21],
        [26, 26, 26, ..., 20, 20, 20]], dtype=uint8),
 array([[40, 40, 40, ..., 34, 34, 34],
        [39, 39, 39, ..., 33, 33, 33],
        [38, 38, 38, ..., 33, 33, 33],
        ...,
        [47, 47, 47, ..., 39, 39, 39],
        [46, 46, 46, ..., 39, 38, 38],
        [45, 45, 45, ..., 38, 38, 38]], dtype=uint8)]

In [12]:
[ dyssynchonia_analysis.analyze_image(image, min_dist_between_maxs, ancho_corte) for image in images ]

[{'image': {'intensities': array([3178, 3294, 3436, 3602, 3797, 4008, 4247, 4490, 4767, 5034, 5323,
          5588, 5845, 6125, 6387, 6610, 6820, 7007, 7157, 7330, 7475, 7612,
          7730, 7814, 7858, 7900, 7909, 7908, 7915, 7904, 7905, 7883, 7827,
          7807, 7783, 7738, 7692, 7658, 7579, 7499, 7396, 7302, 7238, 7174,
          7097, 7022, 6960, 6865, 6802, 6718, 6644, 6535, 6443, 6326, 6231,
          6138, 6032, 5936, 5838, 5745, 5636, 5530, 5420, 5293, 5161, 5026,
          4893, 4774, 4657, 4539, 4450, 4332, 4225, 4131, 4022, 3903, 3807,
          3692, 3580, 3492, 3398, 3321, 3254, 3197, 3125, 3072, 3018, 2977,
          2966, 2959, 2974, 3004, 3047, 3116, 3199, 3311, 3464, 3621, 3793,
          3988, 4207, 4445, 4715, 4990, 5281, 5557, 5836, 6109, 6366, 6619,
          6847, 7078, 7262, 7452, 7591, 7729, 7828, 7928, 7987, 8035, 8050,
          8035, 8039, 8006, 7998, 7960, 7930, 7881, 7837, 7774, 7729, 7690,
          7618, 7558, 7494, 7408, 7332, 7275, 7195, 7120, 7034, 